# import necessaries library for setting up the environment

In [1]:
#main library for data analysis
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
#library for json file
import json 
#convert address to latitude, longitude values 
from geopy.geocoders import Nominatim
#library for handle request
import requests
#transform json file into pandas df
from pandas.io.json import json_normalize
#matplotlib for plotting module
import matplotlib.cm as cm
import matplotlib.colors as colors
#slkearn for KMeans Cluster
from sklearn.cluster import KMeans
#folium for mapping 
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    altair-3.1.0               |           py36_0         724 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.2 MB

The following NEW packages will be 

# web scrapping 

In [3]:
#import Beautifulsoup for scrapping wiki
from bs4 import BeautifulSoup

In [6]:
#store wiki page in url
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#open url with beautiful soup
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')

In [12]:
print(soup)

<!DOCTYPE html>

<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of postal codes of Canada: M - Wikipedia</title>
<script>document.documentElement.className=document.documentElement.className.replace(/(^|\s)client-nojs(\s|$)/,"$1client-js$2");RLCONF={"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":906439794,"wgRevisionId":906439794,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgBreakFrames":!1,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","

#### if you print soup you get text in html so we needed to scrape them into table with 3 columns
#### Create a [ ] so we can stored data within 3 cloumns 

In [24]:
#create an empty list to stored 
postalCode_list = [] 
borough_list = [] 
neighborhood_list = []
#create for loop to find tag<tr> and tag<td>
for i in soup.find('table').find_all('tr'):
    x = i.find_all('td')
    if (len(x) > 0):
        postalCode_list.append(x[0].text)
        borough_list.append(x[1].text)
        neighborhood_list.append(x[2].text.rstrip('\n'))
print( 'This is postalCode_list',postalCode_list[0:5])
print('This is borough_list',borough_list[0:5])
print('This is neihborhood_list',neighborhood_list[0:5])

This is postalCode_list ['M1A', 'M2A', 'M3A', 'M4A', 'M5A']
This is borough_list ['Not assigned', 'Not assigned', 'North York', 'North York', 'Downtown Toronto']
This is neihborhood_list ['Not assigned', 'Not assigned', 'Parkwoods', 'Victoria Village', 'Harbourfront']


#### Let's put these in DataFrame

In [41]:
df = [('PostalCode', postalCode_list), ('Borough', borough_list),('Neighborhood', 
       neighborhood_list)]

In [44]:
df1 = pd.DataFrame.from_dict(dict(df))
df1.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### As we can see there are 'Not assigned' in Borough, Neighborhood columns, we don't want to deal with missing valued

In [50]:
df1_dropna = df1[df1['Borough'] != 'Not assigned'].reset_index(drop = True)
df1_dropna.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


#### From the instruction,
#### 'More than one neighborhood can exist in one postal code area. 
#### For example, in the table on the Wikipedia page, 
#### you will notice that M5A is listed twice and has two neighborhoods: 
#### Harbourfront and Regent Park. These two rows will be combined 
#### into one row with the neighborhoods separated with a comma 
#### as shown in row 11 in the above table
##### let's grouped neighbood by Postal and Borough

In [64]:
df1_dropna_grouped = df1_dropna.groupby(['PostalCode','Borough'],
as_index = False).agg(lambda x: ','.join(x))
df1_dropna_grouped

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


#### According to the instruction,
#### 'If a cell has a borough but a Not assigned neighborhood, 
#### then the neighborhood will be the same as the borough. 
#### So for the 9th cell in the table on the Wikipedia page, 
#### the value of the Borough and the Neighborhood columns will be Queen's Park.'
#### 85th has 'Not assigned' valued in Neighborhood, so we decied to assign <br><br> the value to be the same as Borough

In [93]:
neighborhood_na = df1_dropna_grouped['Neighborhood'] == 'Not assigned'
#located the 'Not assigned' and assign to the same value as Borough
df1_dropna_grouped.loc[neighborhood_na, 'Neighborhood'] = df1_dropna_grouped.loc[neighborhood_na, 'Borough']
df1_dropna_grouped[75:86]

,PostalCode,Borough,Neighborhood
75,M6G,Downtown Toronto,Christie
76,M6H,West Toronto,"Dovercourt Village,Dufferin"
77,M6J,West Toronto,"Little Portugal,Trinity"
78,M6K,West Toronto,"Brockton,Exhibition Place,Parkdale Village"
79,M6L,North York,"Downsview,North Park,Upwood Park"
80,M6M,York,"Del Ray,Keelesdale,Mount Dennis,Silverthorn"
81,M6N,York,"The Junction North,Runnymede"
82,M6P,West Toronto,"High Park,The Junction South"
83,M6R,West Toronto,"Parkdale,Roncesvalles"
84,M6S,West Toronto,"Runnymede,Swansea"


In [95]:
Cleaned_df = df1_dropna_grouped
Cleaned_df.shape

(103, 3)